<a href="https://colab.research.google.com/github/rkwi/master-thesis/blob/master/code/gauge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files

import platform, sys
python_version=platform.python_version()
from distutils.version import LooseVersion, StrictVersion

if ( LooseVersion(python_version) < LooseVersion("3.0.0")):
    print("Python3 is needed!");
    print("How to fix: Runtime/Change_runtime_type/Python 3");
    sys.exit()
try:
    from dolfin import *; from mshr import *
except ImportError as e:
    !apt-get install -y -qq software-properties-common python-software-properties module-init-tools
    !add-apt-repository -y ppa:fenics-packages/fenics
    !apt-get update -qq
    !apt install -y --no-install-recommends fenics
    from dolfin import *; from mshr import *
    
import matplotlib.pyplot as plt;
from IPython.display import clear_output, display; import time; import dolfin.common.plotting as fenicsplot 
import time
import numpy as np
import pandas as pd
from fenics import *
import os, sys, shutil

dolfin_version = dolfin.__version__
print ('dolfin version:', dolfin_version)

!rm -rf * # clean up all files
# Useful commands
# Remove an empty folder      : os.rmdir("my_results")
# Remove a folder with files  : shutil.rmtree("results")
# Make a folder               : os.mkdir("my_results")
# Runtime/Change_runtime_type/Python3

In [0]:
def errornorm_L1_vector(u_e, u):
  V = u.function_space()
  mesh = V.mesh()
  degree = V.ufl_element().degree()
  W = VectorFunctionSpace(mesh, 'P', degree + 4)
  u_e_W = interpolate(u_e, W)
  u_W = interpolate(u, W)
  e_W = Function(W)
  e_W.vector()[:] = u_e_W.vector().get_local() - u_W.vector().get_local()
  error = (abs(e_W.sub(0))+abs(e_W.sub(1)))*dx
  return abs(assemble(error))

def errornorm_L1(u_e, u):
  V = u.function_space()
  mesh = V.mesh()
  degree = V.ufl_element().degree()
  W = FunctionSpace(mesh, 'P', degree + 4)
  u_e_W = interpolate(u_e, W)
  u_W = interpolate(u, W)
  e_W = Function(W)
  e_W.vector()[:] = u_e_W.vector().get_local() - u_W.vector().get_local()
  error = abs(e_W)*dx
  return abs(assemble(error))

def errornorm_Linf_vector(u_e, u):
  S = VectorFunctionSpace(u.function_space().mesh(),'P',u.function_space().ufl_element().degree() + 4)
  u_e_interpolated = interpolate(u_e,S)
  u_interpolated = interpolate(u,S)
  temp = u_e_interpolated.vector() - u_interpolated.vector()
  return norm(temp,'linf')

def errornorm_Linf(u_e, u):
  S = FunctionSpace(u.function_space().mesh(),'P',u.function_space().ufl_element().degree() + 4)
  u_e_interpolated = interpolate(u_e,S)
  u_interpolated = interpolate(u,S)
  temp = u_e_interpolated.vector() - u_interpolated.vector()
  return norm(temp,'linf')

def create_error_table(N_list, L1_error, L2_error, Linf_error):
  row = len(L1_error)
  data_array = np.zeros((row,6))
  for i in range(row):
    for j in range(6):
      if j == 0:
        data_array[i][j] = L1_error[i]
      elif j == 1 and i < row - 1:
        data_array[i][j] = np.log2(L1_error[i]/L1_error[i+1])
      elif j == 2:
        data_array[i][j] = L2_error[i]
      elif j == 3 and i < row - 1:
        data_array[i][j] = np.log2(L2_error[i]/L2_error[i+1])
      elif j == 4:
        data_array[i][j] = Linf_error[i]
      elif j == 5 and i < row - 1:
        data_array[i][j] = np.log2(Linf_error[i]/Linf_error[i+1])
  table = pd.DataFrame(data_array)
  table.set_index([[str('1/' + str(N)) for N in N_list]],inplace=True)
  table.columns = [['L1','L1_rate','L2','L2_rate','Linf','Linf_rate']]
  return table

In [0]:
import time, math

N_list = [16,32,64,128]
u_list = []
u_exact_list = []
phi_list = []
phi_exact_list = []
for N in N_list:
  T = 0.5
  dt = 1.0/(2*N)
  k = Constant(dt)
  num_steps = int(T/dt)
  interpolation_degree = 3
  R = 1
  

  f = Expression(('(2*sin(t)*pow(sin(pi*x[0]), 2)*sin(pi*x[1]) + pi*sin(t)*sin(pi*x[0]) - 16*pow(pi, 2)*pow(sin(pi*x[0]), 2)*sin(pi*x[1])*cos(t) - 2*pow(pi, 3)*sin(pi*x[0])*cos(t) + 4*pow(pi, 2)*sin(pi*x[1])*cos(t))*cos(pi*x[1])'
                  ,'(-2*sin(t)*sin(pi*x[0])*pow(sin(pi*x[1]), 2) + pi*sin(t)*sin(pi*x[1]) + 16*pow(pi, 2)*sin(pi*x[0])*pow(sin(pi*x[1]), 2)*cos(t) - 4*pow(pi, 2)*sin(pi*x[0])*cos(t) - 2*pow(pi, 3)*sin(pi*x[1])*cos(t))*cos(pi*x[0])')
                 ,degree=interpolation_degree,t=0)
  
  f0 = Expression(('(2*sin(t)*pow(sin(pi*x[0]), 2)*sin(pi*x[1]) + pi*sin(t)*sin(pi*x[0]) - 16*pow(pi, 2)*pow(sin(pi*x[0]), 2)*sin(pi*x[1])*cos(t) - 2*pow(pi, 3)*sin(pi*x[0])*cos(t) + 4*pow(pi, 2)*sin(pi*x[1])*cos(t))*cos(pi*x[1])'
                  ,'(-2*sin(t)*sin(pi*x[0])*pow(sin(pi*x[1]), 2) + pi*sin(t)*sin(pi*x[1]) + 16*pow(pi, 2)*sin(pi*x[0])*pow(sin(pi*x[1]), 2)*cos(t) - 4*pow(pi, 2)*sin(pi*x[0])*cos(t) - 2*pow(pi, 3)*sin(pi*x[1])*cos(t))*cos(pi*x[0])')
                 ,degree=interpolation_degree,t=0)
  
  u_exact = Expression(('-pow(sin(pi*x[0]), 2)*sin(2*pi*x[1])*cos(t)'
                        ,'sin(2*pi*x[0])*pow(sin(pi*x[1]), 2)*cos(t)')
                       ,degree=interpolation_degree,t=0)

  phi_exact = Expression('cos(t)*cos(pi*x[0])*cos(pi*x[1])',degree=interpolation_degree,t=0)

  mesh = UnitSquareMesh(N,N)
  V = VectorFunctionSpace(mesh, 'P', 1)
  Q = FunctionSpace(mesh, 'P', 1)

  u = TrialFunction(V)
  m = TrialFunction(V)
  phi = TrialFunction(Q)

  v = TestFunction(V)
  q = TestFunction(Q)

  u1 = Function(V)
  m0 = Function(V)
  m1 = Function(V)
  phi0 = Function(Q)
  phi1 = Function(Q)
  phi2 = Function(Q)

  top_bottom = 'near(x[1],0) || near(x[1],1)'
  walls = 'near(x[0],0) || near(x[0],1)'
  center = 'near(x[0],0.5) && near(x[1], 0.5)'
  
  def boundary_function(x, on_boundary):
    return on_boundary
  
  def boundary_condition_m_second(phi0, phi1):
    bc_top_bottom_x = DirichletBC(V.sub(0),project(2*phi1.dx(0) - phi0.dx(0),Q),top_bottom)
    bc_top_bottom_y = DirichletBC(V.sub(1), 0, top_bottom)
    bc_walls_x = DirichletBC(V.sub(0), 0, walls)
    bc_walls_y = DirichletBC(V.sub(1), project(2*phi1.dx(1) - phi0.dx(1),Q), walls)
    return [bc_top_bottom_x, bc_top_bottom_y, bc_walls_x, bc_walls_y]

  bcm = boundary_condition_m_second(phi0, phi1)
  bcphi = [DirichletBC(Q, 0, center, 'pointwise')]
  bcu = [DirichletBC(V, (0,0), boundary_function)]

  F1 = (1.0/k)*inner(m-m0,v)*dx + 0.5*(1/R)*inner(grad(m) + grad(m0),grad(v))*dx-0.5*inner(f + f0,v)*dx
  a1 = lhs(F1)
  L1 = rhs(F1)

  a2 = inner(grad(phi),grad(q))*dx
  L2 = -div(m1)*q*dx

  a3 = inner(u,v)*dx
  L3 = inner(m1-grad(phi2),v)*dx

  A1 = assemble(a1)
  A2 = assemble(a2)
  A3 = assemble(a3)

  t = 0
  for n in range(num_steps):

    t += dt
    f.t = t
    f0.t = t - dt
    u_exact.t = t
    phi_exact.t = t

    b1 = assemble(L1)
    [bc.apply(A1, b1) for bc in bcm]
    solve(A1, m1.vector(), b1)

    b2 = assemble(L2)
    [bc.apply(A2, b2) for bc in bcphi]
    solve(A2, phi2.vector(), b2)

    b3 = assemble(L3)
    [bc.apply(A3,b3) for bc in bcu]
    solve(A3, u1.vector(), b3)

    
    m0.assign(m1)
   
    if n == 0:
      phi1.assign(phi2)
    else:
      phi0.assign(phi1)
      phi1.assign(phi2)
    bcm = boundary_condition_m_second(phi0, phi1)
    
  print('N =',N)
  u_list.append(u1)
  u_exact_list.append(u_exact)
  phi_list.append(phi2)
  phi_exact_list.append(phi_exact)
  
u_error_L2_list = []
u_error_L1_list = []
phi_error_L2_list = []
phi_error_L1_list = []
u_error_Linf_list = []
phi_error_Linf_list = []

for i in range(len(u_list)):
  
  u_error_L2_list.append(errornorm(u_exact_list[i],u_list[i],'L2'))
  phi_error_L2_list.append(errornorm(phi_exact_list[i],phi_list[i],'L2'))
  
  u_error_Linf_list.append(errornorm_Linf_vector(u_exact_list[i],u_list[i]))
  phi_error_Linf_list.append(errornorm_Linf(phi_exact_list[i],phi_list[i]))
  
  u_error_L1_list.append(errornorm_L1_vector(u_exact_list[i],u_list[i]))
  phi_error_L1_list.append(errornorm_L1(phi_exact_list[i],phi_list[i]))


In [0]:
u_table = create_error_table(N_list,u_error_L1_list,u_error_L2_list,u_error_Linf_list)

In [9]:
u_table

,L1,L1_rate,L2,L2_rate,Linf,Linf_rate
1/16,0.040930,1.901805,0.036322,1.949091,0.063090,1.547581
1/32,0.010953,1.956065,0.009407,1.961266,0.021582,0.976636
1/64,0.002823,1.986180,0.002416,1.971454,0.010967,0.997604
1/128,0.000713,0.000000,0.000616,0.000000,0.005493,0.000000


In [0]:
phi_table = create_error_table(N_list,phi_error_L1_list,phi_error_L2_list,phi_error_Linf_list)

In [11]:
phi_table

,L1,L1_rate,L2,L2_rate,Linf,Linf_rate
1/16,0.025327,2.091353,0.032131,2.078157,0.087333,1.740586
1/32,0.005943,1.886738,0.007609,1.880651,0.026134,1.641399
1/64,0.001607,1.859347,0.002066,1.856672,0.008377,1.666315
1/128,0.000443,0.000000,0.000571,0.000000,0.002639,0.000000
